<a href="https://colab.research.google.com/github/Pranavh-2004/AI_Media_Detection-Synapse2.0/blob/main/Working_Product.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import tensorflow as tf
import zipfile
import os
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_all_data.zip
zipref=zipfile.ZipFile("10_food_classes_all_data.zip")
zipref.extractall()
zipref.close()
!pip install imagehash

--2024-04-20 05:12:33--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_all_data.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.124.207, 142.250.152.207, 142.250.128.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.124.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 519183241 (495M) [application/zip]
Saving to: ‘10_food_classes_all_data.zip.2’

10_food_classes_all 100%[===================>] 495.13M  70.3MB/s    in 7.6s    

2024-04-20 05:12:41 (64.9 MB/s) - ‘10_food_classes_all_data.zip.2’ saved [519183241/519183241]



In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


tf.random.set_seed(42)

# Define ImageDataGenerators for training and testing

train_datagen_augmented = ImageDataGenerator(
    rotation_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    height_shift_range=0.2,
    width_shift_range=0.2

)

test_datagen = ImageDataGenerator(
    #rescale=1./255

)
train_datagen = ImageDataGenerator(
    #rescale=1./255

)

# Flow training images in batches using the generator
train_data_augmented = train_datagen_augmented.flow_from_directory(
    '10_food_classes_all_data/train/',
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    seed=42,
    shuffle=True
)

# Flow validation images in batches using the generator
test_data = test_datagen.flow_from_directory(
    '10_food_classes_all_data/test/',
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    seed=42
)
train_data=train_datagen.flow_from_directory(
    '10_food_classes_all_data/train/',
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    seed=42,
    shuffle=True
)

Found 7500 images belonging to 10 classes.
Found 2500 images belonging to 10 classes.
Found 7500 images belonging to 10 classes.


In [9]:
import tensorflow as tf
import numpy as np

base_model=tf.keras.applications.EfficientNetB4(include_top=False)
base_model.trainable=False #we are taking pre trained model so we shouldnt train it and disturb its accuracy

inputs=tf.keras.layers.Input(shape=(128,128,3))
x=base_model(inputs)
x=tf.keras.layers.GlobalAveragePooling2D()(x)
outputs=tf.keras.layers.Dense(10,activation='softmax')(x)
model=tf.keras.models.Model(inputs,outputs)
model.summary()
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
               optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
               metrics=['accuracy'])


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 efficientnetb4 (Functional  (None, None, None, 1792   17673823  
 )                           )                                   
                                                                 
 global_average_pooling2d_1  (None, 1792)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_1 (Dense)             (None, 10)                17930     
                                                                 
Total params: 17691753 (67.49 MB)
Trainable params: 17930 (70.04 KB)
Non-trainable params: 17673823 (67.42 MB)
_________________________________________________________________


In [10]:
model.fit(train_data_augmented,epochs=3,validation_data=test_data,steps_per_epoch=len(train_data),validation_steps=int(0.2*len(test_data)))

Epoch 1/3
235/235 [==============================] - 628s 3s/step - loss: 1.2658 - accuracy: 0.5995 - val_loss: 0.7093 - val_accuracy: 0.7688
Epoch 2/3
235/235 [==============================] - 597s 3s/step - loss: 0.9416 - accuracy: 0.6967 - val_loss: 0.6451 - val_accuracy: 0.8042
Epoch 3/3
235/235 [==============================] - 582s 2s/step - loss: 0.8699 - accuracy: 0.7135 - val_loss: 0.6133 - val_accuracy: 0.8062


In [11]:
base_model.trainable=True
for i,layer in enumerate(base_model.layers[:-10]):
  layer.trainable=False

In [12]:
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
               optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),#new learning rate after changing layers=old learning rate/10
               metrics=['accuracy'])
model.fit(train_data_augmented,epochs=6,validation_data=test_data,steps_per_epoch=len(train_data),validation_steps=int(0.2*len(test_data)),initial_epoch=3)

Epoch 4/6
235/235 [==============================] - 654s 3s/step - loss: 0.8686 - accuracy: 0.7116 - val_loss: 0.6204 - val_accuracy: 0.7875
Epoch 5/6
235/235 [==============================] - 611s 3s/step - loss: 0.7839 - accuracy: 0.7437 - val_loss: 0.5508 - val_accuracy: 0.8125
Epoch 6/6
235/235 [==============================] - 612s 3s/step - loss: 0.7072 - accuracy: 0.7668 - val_loss: 0.4781 - val_accuracy: 0.8167


In [13]:
# Perform inference on sample images from the training data
images, labels = next(train_data)  # Get one batch for inference
predictions = model.predict(images)

# Convert predictions to desired format
object_positions_list = []
for pred in predictions:
    object_positions = []
    for i in range(0, len(pred), 4):  # Assuming each object has 4 coordinates (a, b, c, d)
        if i + 4 <= len(pred):  # Check if there are enough values for unpacking
            a, b, c, d = pred[i:i+4]
            object_positions.append({'x': [[a, b], [c, d]]})
        else:
            break
    object_positions_list.append({'objects': object_positions})

print(object_positions_list)

1/1 [==============================] - 6s 6s/step
[{'objects': [{'x': [[0.1914132, 0.56806844], [0.01987432, 0.06664723]]}, {'x': [[0.002473449, 0.00081576436], [0.113705255, 0.029870503]]}]}, {'objects': [{'x': [[0.06651431, 0.078465216], [0.32503846, 0.01978449]]}, {'x': [[0.0012009578, 0.0017722325], [0.0073262937, 0.14076643]]}]}, {'objects': [{'x': [[0.015676245, 0.0013215535], [0.9181736, 0.007259379]]}, {'x': [[0.0045528742, 0.00029989926], [0.0038643705, 0.0064367764]]}]}, {'objects': [{'x': [[0.00026248273, 0.0021917704], [0.0014579231, 0.002119934]]}, {'x': [[0.003878405, 0.007929086], [0.001305932, 0.0004167375]]}]}, {'objects': [{'x': [[0.0017771195, 0.0029942212], [0.0058182576, 0.01611626]]}, {'x': [[0.011431759, 0.003601527], [0.016552014, 0.0005920256]]}]}, {'objects': [{'x': [[0.0034851483, 0.00042670438], [0.0023543178, 0.009210413]]}, {'x': [[0.95156664, 0.00024417482], [3.6481848e-05, 0.020524535]]}]}, {'objects': [{'x': [[0.006710948, 0.0014575939], [0.0028148517, 

In [14]:
from __future__ import absolute_import, division, print_function

import sys

import numpy
from PIL import Image, ImageFilter
try:
	ANTIALIAS = Image.Resampling.LANCZOS
except AttributeError:
	# deprecated in pillow 10
	# https://pillow.readthedocs.io/en/stable/deprecations.html
	ANTIALIAS = Image.ANTIALIAS

__version__ = '4.3.1'

"""
You may copy this file, if you keep the copyright information below:


Copyright (c) 2013-2022, Johannes Buchner
https://github.com/JohannesBuchner/imagehash

All rights reserved.

Redistribution and use in source and binary forms, with or without
modification, are permitted provided that the following conditions are
met:

Redistributions of source code must retain the above copyright
notice, this list of conditions and the following disclaimer.

Redistributions in binary form must reproduce the above copyright
notice, this list of conditions and the following disclaimer in the
documentation and/or other materials provided with the distribution.

THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS
IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED
TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A
PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT
HOLDER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL,
SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT
LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE,
DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY
THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT
(INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE
OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.

"""


def _binary_array_to_hex(arr):
	"""
	internal function to make a hex string out of a binary array.
	"""
	bit_string = ''.join(str(b) for b in 1 * arr.flatten())
	width = int(numpy.ceil(len(bit_string) / 4))
	return '{:0>{width}x}'.format(int(bit_string, 2), width=width)


class ImageHash:
	"""
	Hash encapsulation. Can be used for dictionary keys and comparisons.
	"""

	def __init__(self, binary_array):
		# type: (NDArray) -> None
		self.hash = binary_array  # type: NDArray

	def __str__(self):
		return _binary_array_to_hex(self.hash.flatten())

	def __repr__(self):
		return repr(self.hash)

	def __sub__(self, other):
		# type: (ImageHash) -> int
		if other is None:
			raise TypeError('Other hash must not be None.')
		if self.hash.size != other.hash.size:
			raise TypeError('ImageHashes must be of the same shape.', self.hash.shape, other.hash.shape)
		return numpy.count_nonzero(self.hash.flatten() != other.hash.flatten())

	def __eq__(self, other):
		# type: (object) -> bool
		if other is None:
			return False
		return numpy.array_equal(self.hash.flatten(), other.hash.flatten())  # type: ignore

	def __ne__(self, other):
		# type: (object) -> bool
		if other is None:
			return False
		return not numpy.array_equal(self.hash.flatten(), other.hash.flatten())  # type: ignore

	def __hash__(self):
		# this returns a 8 bit integer, intentionally shortening the information
		return sum([2**(i % 8) for i, v in enumerate(self.hash.flatten()) if v])

	def __len__(self):
		# Returns the bit length of the hash
		return self.hash.size


# dynamic code for typing
try:
	# specify allowed values if possible (py3.8+)
	from typing import Literal
	WhashMode = Literal['haar', 'db4']  # type: ignore
except ImportError:
	WhashMode = str  # type: ignore

try:
	# enable numpy array typing (py3.7+)
	import numpy.typing
	NDArray = numpy.typing.NDArray[numpy.bool_]
except (AttributeError, ImportError):
	NDArray = list  # type: ignore

# type of Callable
if sys.version_info >= (3, 3):
	if sys.version_info >= (3, 9, 0) and sys.version_info <= (3, 9, 1):
		# https://stackoverflow.com/questions/65858528/is-collections-abc-callable-bugged-in-python-3-9-1
		from typing import Callable
	else:
		from collections.abc import Callable
	try:
		MeanFunc = Callable[[NDArray], float]
		HashFunc = Callable[[Image.Image], ImageHash]
	except TypeError:
		MeanFunc = Callable  # type: ignore
		HashFunc = Callable  # type: ignore
# end of dynamic code for typing


def hex_to_hash(hexstr):
	# type: (str) -> ImageHash
	"""
	Convert a stored hash (hex, as retrieved from str(Imagehash))
	back to a Imagehash object.

	Notes:
	1. This algorithm assumes all hashes are either
			bidimensional arrays with dimensions hash_size * hash_size,
			or onedimensional arrays with dimensions binbits * 14.
	2. This algorithm does not work for hash_size < 2.
	"""
	hash_size = int(numpy.sqrt(len(hexstr) * 4))
	# assert hash_size == numpy.sqrt(len(hexstr)*4)
	binary_array = '{:0>{width}b}'.format(int(hexstr, 16), width=hash_size * hash_size)
	bit_rows = [binary_array[i:i + hash_size] for i in range(0, len(binary_array), hash_size)]
	hash_array = numpy.array([[bool(int(d)) for d in row] for row in bit_rows])
	return ImageHash(hash_array)


def hex_to_flathash(hexstr, hashsize):
	# type: (str, int) -> ImageHash
	hash_size = int(len(hexstr) * 4 / (hashsize))
	binary_array = '{:0>{width}b}'.format(int(hexstr, 16), width=hash_size * hashsize)
	hash_array = numpy.array([[bool(int(d)) for d in binary_array]])[-hash_size * hashsize:]
	return ImageHash(hash_array)


def hex_to_multihash(hexstr):
	# type: (str) -> ImageMultiHash
	"""
	Convert a stored multihash (hex, as retrieved from str(ImageMultiHash))
	back to an ImageMultiHash object.

	This function is based on hex_to_hash so the same caveats apply. Namely:

	1. This algorithm assumes all hashes are either
			bidimensional arrays with dimensions hash_size * hash_size,
			or onedimensional arrays with dimensions binbits * 14.
	2. This algorithm does not work for hash_size < 2.
	"""
	split = hexstr.split(',')
	hashes = [hex_to_hash(x) for x in split]
	return ImageMultiHash(hashes)


def old_hex_to_hash(hexstr, hash_size=8):
	# type: (str, int) -> ImageHash
	"""
	Convert a stored hash (hex, as retrieved from str(Imagehash))
	back to a Imagehash object. This method should be used for
	hashes generated by ImageHash up to version 3.7. For hashes
	generated by newer versions of ImageHash, hex_to_hash should
	be used instead.
	"""
	arr = []
	count = hash_size * (hash_size // 4)
	if len(hexstr) != count:
		emsg = 'Expected hex string size of {}.'
		raise ValueError(emsg.format(count))
	for i in range(count // 2):
		h = hexstr[i * 2:i * 2 + 2]
		v = int('0x' + h, 16)
		arr.append([v & 2**i > 0 for i in range(8)])
	return ImageHash(numpy.array(arr))


def average_hash(image, hash_size=8, mean=numpy.mean):
	# type: (Image.Image, int, MeanFunc) -> ImageHash
	"""
	Average Hash computation

	Implementation follows https://www.hackerfactor.com/blog/index.php?/archives/432-Looks-Like-It.html

	Step by step explanation: https://web.archive.org/web/20171112054354/https://www.safaribooksonline.com/blog/2013/11/26/image-hashing-with-python/ # noqa: E501

	@image must be a PIL instance.
	@mean how to determine the average luminescence. can try numpy.median instead.
	"""
	if hash_size < 2:
		raise ValueError('Hash size must be greater than or equal to 2')

	# reduce size and complexity, then convert to grayscale
	image = image.convert('L').resize((hash_size, hash_size), ANTIALIAS)

	# find average pixel value; 'pixels' is an array of the pixel values, ranging from 0 (black) to 255 (white)
	pixels = numpy.asarray(image)
	avg = mean(pixels)

	# create string of bits
	diff = pixels > avg
	# make a hash
	return ImageHash(diff)


def phash(image, hash_size=8, highfreq_factor=4):
	# type: (Image.Image, int, int) -> ImageHash
	"""
	Perceptual Hash computation.

	Implementation follows https://www.hackerfactor.com/blog/index.php?/archives/432-Looks-Like-It.html

	@image must be a PIL instance.
	"""
	if hash_size < 2:
		raise ValueError('Hash size must be greater than or equal to 2')

	import scipy.fftpack
	img_size = hash_size * highfreq_factor
	image = image.convert('L').resize((img_size, img_size), ANTIALIAS)
	pixels = numpy.asarray(image)
	dct = scipy.fftpack.dct(scipy.fftpack.dct(pixels, axis=0), axis=1)
	dctlowfreq = dct[:hash_size, :hash_size]
	med = numpy.median(dctlowfreq)
	diff = dctlowfreq > med
	return ImageHash(diff)


def phash_simple(image, hash_size=8, highfreq_factor=4):
	# type: (Image.Image, int, int) -> ImageHash
	"""
	Perceptual Hash computation.

	Implementation follows https://www.hackerfactor.com/blog/index.php?/archives/432-Looks-Like-It.html

	@image must be a PIL instance.
	"""
	import scipy.fftpack
	img_size = hash_size * highfreq_factor
	image = image.convert('L').resize((img_size, img_size), ANTIALIAS)
	pixels = numpy.asarray(image)
	dct = scipy.fftpack.dct(pixels)
	dctlowfreq = dct[:hash_size, 1:hash_size + 1]
	avg = dctlowfreq.mean()
	diff = dctlowfreq > avg
	return ImageHash(diff)


def dhash(image, hash_size=8):
	# type: (Image.Image, int) -> ImageHash
	"""
	Difference Hash computation.

	following https://www.hackerfactor.com/blog/index.php?/archives/529-Kind-of-Like-That.html

	computes differences horizontally

	@image must be a PIL instance.
	"""
	# resize(w, h), but numpy.array((h, w))
	if hash_size < 2:
		raise ValueError('Hash size must be greater than or equal to 2')

	image = image.convert('L').resize((hash_size + 1, hash_size), ANTIALIAS)
	pixels = numpy.asarray(image)
	# compute differences between columns
	diff = pixels[:, 1:] > pixels[:, :-1]
	return ImageHash(diff)


def dhash_vertical(image, hash_size=8):
	# type: (Image.Image, int) -> ImageHash
	"""
	Difference Hash computation.

	following https://www.hackerfactor.com/blog/index.php?/archives/529-Kind-of-Like-That.html

	computes differences vertically

	@image must be a PIL instance.
	"""
	# resize(w, h), but numpy.array((h, w))
	image = image.convert('L').resize((hash_size, hash_size + 1), ANTIALIAS)
	pixels = numpy.asarray(image)
	# compute differences between rows
	diff = pixels[1:, :] > pixels[:-1, :]
	return ImageHash(diff)


def whash(image, hash_size=8, image_scale=None, mode='haar', remove_max_haar_ll=True):
	# type: (Image.Image, int, int | None, WhashMode, bool) -> ImageHash
	"""
	Wavelet Hash computation.

	based on https://www.kaggle.com/c/avito-duplicate-ads-detection/

	@image must be a PIL instance.
	@hash_size must be a power of 2 and less than @image_scale.
	@image_scale must be power of 2 and less than image size. By default is equal to max
					power of 2 for an input image.
	@mode (see modes in pywt library):
					'haar' - Haar wavelets, by default
					'db4' - Daubechies wavelets
	@remove_max_haar_ll - remove the lowest low level (LL) frequency using Haar wavelet.
	"""
	import pywt
	if image_scale is not None:
		assert image_scale & (image_scale - 1) == 0, 'image_scale is not power of 2'
	else:
		image_natural_scale = 2**int(numpy.log2(min(image.size)))
		image_scale = max(image_natural_scale, hash_size)

	ll_max_level = int(numpy.log2(image_scale))

	level = int(numpy.log2(hash_size))
	assert hash_size & (hash_size - 1) == 0, 'hash_size is not power of 2'
	assert level <= ll_max_level, 'hash_size in a wrong range'
	dwt_level = ll_max_level - level

	image = image.convert('L').resize((image_scale, image_scale), ANTIALIAS)
	pixels = numpy.asarray(image) / 255.

	# Remove low level frequency LL(max_ll) if @remove_max_haar_ll using haar filter
	if remove_max_haar_ll:
		coeffs = pywt.wavedec2(pixels, 'haar', level=ll_max_level)
		coeffs = list(coeffs)
		coeffs[0] *= 0
		pixels = pywt.waverec2(coeffs, 'haar')

	# Use LL(K) as freq, where K is log2(@hash_size)
	coeffs = pywt.wavedec2(pixels, mode, level=dwt_level)
	dwt_low = coeffs[0]

	# Subtract median and compute hash
	med = numpy.median(dwt_low)
	diff = dwt_low > med
	return ImageHash(diff)


def colorhash(image, binbits=3):
	# type: (Image.Image, int) -> ImageHash
	"""
	Color Hash computation.

	Computes fractions of image in intensity, hue and saturation bins:

	* the first binbits encode the black fraction of the image
	* the next binbits encode the gray fraction of the remaining image (low saturation)
	* the next 6*binbits encode the fraction in 6 bins of saturation, for highly saturated parts of the remaining image
	* the next 6*binbits encode the fraction in 6 bins of saturation, for mildly saturated parts of the remaining image

	@binbits number of bits to use to encode each pixel fractions
	"""

	# bin in hsv space:
	intensity = numpy.asarray(image.convert('L')).flatten()
	h, s, v = [numpy.asarray(v).flatten() for v in image.convert('HSV').split()]
	# black bin
	mask_black = intensity < 256 // 8
	frac_black = mask_black.mean()
	# gray bin (low saturation, but not black)
	mask_gray = s < 256 // 3
	frac_gray = numpy.logical_and(~mask_black, mask_gray).mean()
	# two color bins (medium and high saturation, not in the two above)
	mask_colors = numpy.logical_and(~mask_black, ~mask_gray)
	mask_faint_colors = numpy.logical_and(mask_colors, s < 256 * 2 // 3)
	mask_bright_colors = numpy.logical_and(mask_colors, s > 256 * 2 // 3)

	c = max(1, mask_colors.sum())
	# in the color bins, make sub-bins by hue
	hue_bins = numpy.linspace(0, 255, 6 + 1)
	if mask_faint_colors.any():
		h_faint_counts, _ = numpy.histogram(h[mask_faint_colors], bins=hue_bins)
	else:
		h_faint_counts = numpy.zeros(len(hue_bins) - 1)
	if mask_bright_colors.any():
		h_bright_counts, _ = numpy.histogram(h[mask_bright_colors], bins=hue_bins)
	else:
		h_bright_counts = numpy.zeros(len(hue_bins) - 1)

	# now we have fractions in each category (6*2 + 2 = 14 bins)
	# convert to hash and discretize:
	maxvalue = 2**binbits
	values = [min(maxvalue - 1, int(frac_black * maxvalue)), min(maxvalue - 1, int(frac_gray * maxvalue))]
	for counts in list(h_faint_counts) + list(h_bright_counts):
		values.append(min(maxvalue - 1, int(counts * maxvalue * 1. / c)))
	# print(values)
	bitarray = []
	for v in values:
		bitarray += [v // (2**(binbits - i - 1)) % 2**(binbits - i) > 0 for i in range(binbits)]
	return ImageHash(numpy.asarray(bitarray).reshape((-1, binbits)))


class ImageMultiHash:
	"""
	This is an image hash containing a list of individual hashes for segments of the image.
	The matching logic is implemented as described in Efficient Cropping-Resistant Robust Image Hashing
	"""

	def __init__(self, hashes):
		# type: (list[ImageHash]) -> None
		self.segment_hashes = hashes  # type: list[ImageHash]

	def __eq__(self, other):
		# type: (object) -> bool
		if other is None:
			return False
		return self.matches(other)  # type: ignore

	def __ne__(self, other):
		# type: (object) -> bool
		return not self.matches(other)  # type: ignore

	def __sub__(self, other, hamming_cutoff=None, bit_error_rate=None):
		# type: (ImageMultiHash, float | None, float | None) -> float
		matches, sum_distance = self.hash_diff(other, hamming_cutoff, bit_error_rate)
		max_difference = len(self.segment_hashes)
		if matches == 0:
			return max_difference
		max_distance = matches * len(self.segment_hashes[0])
		tie_breaker = 0 - (float(sum_distance) / max_distance)
		match_score = matches + tie_breaker
		return max_difference - match_score

	def __hash__(self):
		return hash(tuple(hash(segment) for segment in self.segment_hashes))

	def __str__(self):
		return ','.join(str(x) for x in self.segment_hashes)

	def __repr__(self):
		return repr(self.segment_hashes)

	def hash_diff(self, other_hash, hamming_cutoff=None, bit_error_rate=None):
		# type: (ImageMultiHash, float | None, float | None) -> tuple[int, int]
		"""
		Gets the difference between two multi-hashes, as a tuple. The first element of the tuple is the number of
		matching segments, and the second element is the sum of the hamming distances of matching hashes.
		NOTE: Do not order directly by this tuple, as higher is better for matches, and worse for hamming cutoff.
		:param other_hash: The image multi hash to compare against
		:param hamming_cutoff: The maximum hamming distance to a region hash in the target hash
		:param bit_error_rate: Percentage of bits which can be incorrect, an alternative to the hamming cutoff. The
		default of 0.25 means that the segment hashes can be up to 25% different
		"""
		# Set default hamming cutoff if it's not set.
		if hamming_cutoff is None:
			if bit_error_rate is None:
				bit_error_rate = 0.25
			hamming_cutoff = len(self.segment_hashes[0]) * bit_error_rate
		# Get the hash distance for each region hash within cutoff
		distances = []
		for segment_hash in self.segment_hashes:
			lowest_distance = min(
				segment_hash - other_segment_hash
				for other_segment_hash in other_hash.segment_hashes
			)
			if lowest_distance > hamming_cutoff:
				continue
			distances.append(lowest_distance)
		return len(distances), sum(distances)

	def matches(self, other_hash, region_cutoff=1, hamming_cutoff=None, bit_error_rate=None):
		# type: (ImageMultiHash, int, float | None, float | None) -> bool
		"""
		Checks whether this hash matches another crop resistant hash, `other_hash`.
		:param other_hash: The image multi hash to compare against
		:param region_cutoff: The minimum number of regions which must have a matching hash
		:param hamming_cutoff: The maximum hamming distance to a region hash in the target hash
		:param bit_error_rate: Percentage of bits which can be incorrect, an alternative to the hamming cutoff. The
		default of 0.25 means that the segment hashes can be up to 25% different
		"""
		matches, _ = self.hash_diff(other_hash, hamming_cutoff, bit_error_rate)
		return matches >= region_cutoff

	def best_match(self, other_hashes, hamming_cutoff=None, bit_error_rate=None):
		# type: (list[ImageMultiHash], float | None, float | None) -> ImageMultiHash
		"""
		Returns the hash in a list which is the best match to the current hash
		:param other_hashes: A list of image multi hashes to compare against
		:param hamming_cutoff: The maximum hamming distance to a region hash in the target hash
		:param bit_error_rate: Percentage of bits which can be incorrect, an alternative to the hamming cutoff.
		Defaults to 0.25 if unset, which means the hash can be 25% different
		"""
		return min(
			other_hashes,
			key=lambda other_hash: self.__sub__(other_hash, hamming_cutoff, bit_error_rate)
		)


def _find_region(remaining_pixels, segmented_pixels):
	"""
	Finds a region and returns a set of pixel coordinates for it.
	:param remaining_pixels: A numpy bool array, with True meaning the pixels are remaining to segment
	:param segmented_pixels: A set of pixel coordinates which have already been assigned to segment. This will be
	updated with the new pixels added to the returned segment.
	"""
	in_region = set()
	not_in_region = set()
	# Find the first pixel in remaining_pixels with a value of True
	available_pixels = numpy.transpose(numpy.nonzero(remaining_pixels))
	start = tuple(available_pixels[0])
	in_region.add(start)
	new_pixels = in_region.copy()
	while True:
		try_next = set()
		# Find surrounding pixels
		for pixel in new_pixels:
			x, y = pixel
			neighbours = [
				(x - 1, y),
				(x + 1, y),
				(x, y - 1),
				(x, y + 1)
			]
			try_next.update(neighbours)
		# Remove pixels we have already seen
		try_next.difference_update(segmented_pixels, not_in_region)
		# If there's no more pixels to try, the region is complete
		if not try_next:
			break
		# Empty new pixels set, so we know whose neighbour's to check next time
		new_pixels = set()
		# Check new pixels
		for pixel in try_next:
			if remaining_pixels[pixel]:
				in_region.add(pixel)
				new_pixels.add(pixel)
				segmented_pixels.add(pixel)
			else:
				not_in_region.add(pixel)
	return in_region


def _find_all_segments(pixels, segment_threshold, min_segment_size):
	"""
	Finds all the regions within an image pixel array, and returns a list of the regions.

	Note: Slightly different segmentations are produced when using pillow version 6 vs. >=7, due to a change in
	rounding in the greyscale conversion.
	:param pixels: A numpy array of the pixel brightnesses.
	:param segment_threshold: The brightness threshold to use when differentiating between hills and valleys.
	:param min_segment_size: The minimum number of pixels for a segment.
	"""
	img_width, img_height = pixels.shape
	# threshold pixels
	threshold_pixels = pixels > segment_threshold
	unassigned_pixels = numpy.full(pixels.shape, True, dtype=bool)

	segments = []
	already_segmented = set()

	# Add all the pixels around the border outside the image:
	already_segmented.update([(-1, z) for z in range(img_height)])
	already_segmented.update([(z, -1) for z in range(img_width)])
	already_segmented.update([(img_width, z) for z in range(img_height)])
	already_segmented.update([(z, img_height) for z in range(img_width)])

	# Find all the "hill" regions
	while numpy.bitwise_and(threshold_pixels, unassigned_pixels).any():
		remaining_pixels = numpy.bitwise_and(threshold_pixels, unassigned_pixels)
		segment = _find_region(remaining_pixels, already_segmented)
		# Apply segment
		if len(segment) > min_segment_size:
			segments.append(segment)
		for pix in segment:
			unassigned_pixels[pix] = False

	# Invert the threshold matrix, and find "valleys"
	threshold_pixels_i = numpy.invert(threshold_pixels)
	while len(already_segmented) < img_width * img_height:
		remaining_pixels = numpy.bitwise_and(threshold_pixels_i, unassigned_pixels)
		segment = _find_region(remaining_pixels, already_segmented)
		# Apply segment
		if len(segment) > min_segment_size:
			segments.append(segment)
		for pix in segment:
			unassigned_pixels[pix] = False

	return segments


def crop_resistant_hash(
	image,  # type: Image.Image
	hash_func=None,  # type: HashFunc
	limit_segments=None,  # type: int | None
	segment_threshold=128,  # type: int
	min_segment_size=500,  # type: int
	segmentation_image_size=300  # type: int
):
	# type: (...) -> ImageMultiHash
	"""
	Creates a CropResistantHash object, by the algorithm described in the paper "Efficient Cropping-Resistant Robust
	Image Hashing". DOI 10.1109/ARES.2014.85
	This algorithm partitions the image into bright and dark segments, using a watershed-like algorithm, and then does
	an image hash on each segment. This makes the image much more resistant to cropping than other algorithms, with
	the paper claiming resistance to up to 50% cropping, while most other algorithms stop at about 5% cropping.

	Note: Slightly different segmentations are produced when using pillow version 6 vs. >=7, due to a change in
	rounding in the greyscale conversion. This leads to a slightly different result.
	:param image: The image to hash
	:param hash_func: The hashing function to use
	:param limit_segments: If you have storage requirements, you can limit to hashing only the M largest segments
	:param segment_threshold: Brightness threshold between hills and valleys. This should be static, putting it between
	peak and through dynamically breaks the matching
	:param min_segment_size: Minimum number of pixels for a hashable segment
	:param segmentation_image_size: Size which the image is resized to before segmentation
	"""
	if hash_func is None:
		hash_func = dhash

	orig_image = image.copy()
	# Convert to gray scale and resize
	image = image.convert('L').resize((segmentation_image_size, segmentation_image_size), ANTIALIAS)
	# Add filters
	image = image.filter(ImageFilter.GaussianBlur()).filter(ImageFilter.MedianFilter())
	pixels = numpy.array(image).astype(numpy.float32)

	segments = _find_all_segments(pixels, segment_threshold, min_segment_size)

	# If there are no segments, have 1 segment including the whole image
	if not segments:
		full_image_segment = {(0, 0), (segmentation_image_size - 1, segmentation_image_size - 1)}
		segments.append(full_image_segment)

	# If segment limit is set, discard the smaller segments
	if limit_segments:
		segments = sorted(segments, key=lambda s: len(s), reverse=True)[:limit_segments]

	# Create bounding box for each segment
	hashes = []
	for segment in segments:
		orig_w, orig_h = orig_image.size
		scale_w = float(orig_w) / segmentation_image_size
		scale_h = float(orig_h) / segmentation_image_size
		min_y = min(coord[0] for coord in segment) * scale_h
		min_x = min(coord[1] for coord in segment) * scale_w
		max_y = (max(coord[0] for coord in segment) + 1) * scale_h
		max_x = (max(coord[1] for coord in segment) + 1) * scale_w
		# Compute robust hash for each bounding box
		bounding_box = orig_image.crop((min_x, min_y, max_x, max_y))
		hashes.append(hash_func(bounding_box))


	return ImageMultiHash(hashes)

In [15]:
import numpy as np
from PIL import Image

# Load and preprocess the image
def load_and_preprocess_image(image_path, target_size):
    image = Image.open(image_path)
    image = image.resize(target_size)  # Resize the image to target size
    image = np.array(image) / 255.0     # Normalize pixel values to [0, 1]
    image = image[np.newaxis, ...]      # Add batch dimension
    return image

# Define target size for resizing the image
target_size = (224, 224)  # Adjust according to your model's input shape

# Load and preprocess the image
image = load_and_preprocess_image('/content/3_doremon.jpeg', target_size)

# Predict using the model
predictions = model.predict(image)

# Convert predictions to desired format
object_positions_list = []
for j in range(len(predictions)):
    object_positions = []
    for i in range(0, len(pred), 4):  # Assuming each object has 4 coordinates (a, b, c, d)
        if i + 4 <= len(pred):  # Check if there are enough values for unpacking
            a, b, c, d = pred[i:i+4]
            object_positions.append({j: [[a, b], [c, d]]})
        else:
            break
    object_positions_list += object_positions

print(object_positions_list)

1/1 [==============================] - 4s 4s/step
[{0: [[0.017213995, 0.002100105], [0.0018967421, 0.00093349523]]}, {0: [[0.00041720908, 0.0017323347], [0.0012486514, 0.9723432]]}]


In [16]:
from PIL import Image
import imagehash

def hash_to_binary(hash_str):
    return ''.join(format(ord(char), '08b') for char in hash_str)

def embed_hash(image_path, hash_binary, top_left, bottom_right):
    image = Image.open(image_path)
    pixels = image.load()

    # Calculate center point of the bounding box
    center_x = (top_left[0] + bottom_right[0]) // 2
    center_y = (top_left[1] + bottom_right[1]) // 2

    hash_index = 0
    for i in range(int(top_left[0]), int(bottom_right[0])):
        for j in range(top_left[1], bottom_right[1]):
            r, g, b = pixels[i, j]

            if hash_index < len(hash_binary):
                r = (r & 0xFE) | int(hash_binary[hash_index])
                hash_index += 1
            if hash_index < len(hash_binary):
                g = (g & 0xFE) | int(hash_binary[hash_index])
                hash_index += 1
            if hash_index < len(hash_binary):
                b = (b & 0xFE) | int(hash_binary[hash_index])
                hash_index += 1

            pixels[i, j] = (r, g, b)

    image.save("embedded.jpeg")

# Generate hash
hash_str = imagehash.average_hash(Image.open('/content/3_doremon.jpeg'))
hash_binary = hash_to_binary(str(hash_str))

# Define bounding box coordinates (top-left and bottom-right)
# Replace these with your object's coordinates
for i in object_positions_list:
  for j in i.values():
    [[a,b],[c,d]]= j
top_left = (a, b)   # (x1, y1)
bottom_right = (c, d)   # (x2, y2)
#print(a,b,c,d)

# Embed hash into image at the center of the object
embed_hash('embedded.jpeg', hash_binary, top_left, bottom_right)

In [19]:
def final(image_path):
    # Load and preprocess the image
    image = load_and_preprocess_image(image_path, (128, 128))

    # Predict using the model
    predictions = model.predict(image)

    # Convert predictions to desired format
    object_positions_list = []
    for pred in predictions:
        object_positions = []
        for i in range(0, len(pred), 4):  # Assuming each object has 4 coordinates (a, b, c, d)
            if i + 4 <= len(pred):  # Check if there are enough values for unpacking
                a, b, c, d = pred[i:i+4]
                object_positions.append({'x': [[a, b], [c, d]]})
            else:
                break
        object_positions_list.append({'objects': object_positions})

    # Assuming you have these variables defined elsewhere
    # Generate hash
    hash_str = imagehash.average_hash(Image.open('/content/3_doremon.jpeg'))
    hash_binary = hash_to_binary(str(hash_str))
    #top_left = top_left   # Top-left coordinate of the object
    #bottom_right = bottom_right # Bottom-right coordinate of the object



    # Embed hash into the image
    embed_hash(image_path, hash_binary, top_left, bottom_right)

    # Compute the average hash of the image
    otherhash = imagehash.average_hash(Image.open(image_path))
    otherhash1 = imagehash.average_hash(Image.open('embedded.jpeg'))

    return otherhash, otherhash1

otherhash, otherhash1 = final('/content/3_doremon.jpeg')
print(otherhash)
print(otherhash1)
print(otherhash1 == otherhash)
print(otherhash1 - otherhash)

1/1 [==============================] - 0s 104ms/step
000000ffb7ffff4f
000000ffb7ffff4f
True
0


In [34]:
!pip install pyngrok

from pyngrok import ngrok

# Provide your ngrok authtoken (replace '<your_auth_token>' with your actual token)
ngrok.set_auth_token('2fMDCd00rKWzU0mfFkTbiZBor1J_3PUV9HMREoPU9qRbeStBn')



In [35]:
# Assuming your Flask app is running on port 5000
ngrok_tunnel = ngrok.connect(addr='5000')

In [36]:
public_url = ngrok_tunnel.public_url
print("Public URL:", public_url)

Public URL: https://1809-34-16-24-115.ngrok-free.app


In [37]:
# !pip install flask pyngrok

# from flask import Flask
# from pyngrok import ngrok

# app = Flask(__name__)

# @app.route('/')
# def index():
#     return 'Hello from Flask!'

# # Start ngrok when the app is run
# ngrok_tunnel = ngrok.connect(addr='5000')

# print("Public URL:", ngrok_tunnel.public_url)

# if __name__ == '__main__':
#     app.run()

Public URL: https://4a58-34-16-24-115.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [20/Apr/2024 08:47:37] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Apr/2024 08:47:37] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [20/Apr/2024 08:48:28] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Apr/2024 08:48:29] "GET /favicon.ico HTTP/1.1" 404 -


In [1]:
# Define Flask application
from flask import Flask, render_template, request, redirect, url_for
from PIL import Image
import numpy as np
import imagehash
from io import BytesIO
from flask_ngrok import run_with_ngrok

app = Flask(__name__)

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/upload', methods=['POST'])
def upload():
    if 'file' not in request.files:
        return redirect(request.url)

    file = request.files['file']

    if file.filename == '':
        return redirect(request.url)

    # Read image file
    image = Image.open(file)
    # Preprocess image
    image = image.resize((128, 128))
    image = np.array(image)
    # Compute image hash
    hash_str = imagehash.average_hash(Image.fromarray(image))
    hash_binary = hash_str.hash.flatten().tobytes()

    # Convert hash to hexadecimal string
    hash_hex = hash_str.hex()

    # Embed hash into the image
    embedded_image = Image.open(BytesIO(file.read()))
    embedded_image.save('embedded_image.jpeg')

    return render_template('result.html', hash_hex=hash_hex)

# Define HTML templates as strings
index_html = """
<!DOCTYPE html>
<html>
<head>
    <title>Upload Image</title>
</head>
<body>
    <h1>Upload Image</h1>
    <form action="/upload" method="post" enctype="multipart/form-data">
        <input type="file" name="file" accept="image/*">
        <input type="submit" value="Upload">
    </form>
</body>
</html>
"""

result_html = """
<!DOCTYPE html>
<html>
<head>
    <title>Result</title>
</head>
<body>
    <h1>Image Processing Result</h1>
    <p>Hash code: {{ hash_hex }}</p>
    <img src="/static/embedded_image.jpeg" alt="Embedded Image">
</body>
</html>
"""

# Install dependencies
!pip install flask pillow

# Create HTML template files
with open('index.html', 'w') as file:
    file.write(index_html)

with open('result.html', 'w') as file:
    file.write(result_html)

# Create directory for static files and an empty image file
!mkdir static
!touch static/embedded_image.jpeg

# Start ngrok when the app is run
run_with_ngrok(app)
app.run()


mkdir: cannot create directory ‘static’: File exists
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
Exception in thread Thread-10:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connection.py", line 203, in _new_conn
    sock = connection.create_connection(
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py", line 791, in urlopen
    response = self._make_request(
  File "/usr/local/lib/python3.10/dist-packages/urll

In [4]:
# Define Flask application
from flask import Flask, render_template, request, redirect
from PIL import Image
import numpy as np
import imagehash
from io import BytesIO
from pyngrok import ngrok
import os

app = Flask(__name__)

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/upload', methods=['POST'])
def upload():
    if 'file' not in request.files:
        return redirect(request.url)

    file = request.files['file']

    if file.filename == '':
        return redirect(request.url)

    # Read image file
    image = Image.open(file)
    # Preprocess image
    image = image.resize((128, 128))
    image = np.array(image)
    # Compute image hash
    hash_str = imagehash.average_hash(Image.fromarray(image))
    hash_hex = hash_str.hex()

    # Embed hash into the image
    embedded_image = Image.open(BytesIO(file.read()))
    embedded_image.save('static/embedded_image.jpeg')

    return render_template('result.html', hash_hex=hash_hex)

# Install dependencies
!pip install flask pillow pyngrok

# Define HTML templates as strings
index_html = """
<!DOCTYPE html>
<html>
<head>
    <title>Upload Image</title>
</head>
<body>
    <h1>Upload Image</h1>
    <form action="/upload" method="post" enctype="multipart/form-data">
        <input type="file" name="file" accept="image/*">
        <input type="submit" value="Upload">
    </form>
</body>
</html>
"""

result_html = """
<!DOCTYPE html>
<html>
<head>
    <title>Result</title>
</head>
<body>
    <h1>Image Processing Result</h1>
    <p>Hash code: {{ hash_hex }}</p>
    <img src="/static/embedded_image.jpeg" alt="Embedded Image">
</body>
</html>
"""

# Create templates directory if it doesn't exist
if not os.path.exists('templates'):
    os.makedirs('templates')

# Create HTML template files
with open('templates/index.html', 'w') as file:
    file.write(index_html)

with open('templates/result.html', 'w') as file:
    file.write(result_html)

# Start ngrok tunnel
public_url = ngrok.connect(addr='5000', proto='http')
print('Public URL:', public_url)

# Run Flask app
app.run()


Public URL: NgrokTunnel: "https://6725-34-16-24-115.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [20/Apr/2024 09:04:18] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Apr/2024 09:04:19] "GET /favicon.ico HTTP/1.1" 404 -
ERROR:__main__:Exception on /upload [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 2529, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1825, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1823, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1799, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(*

In [5]:
# Define Flask application
from flask import Flask, render_template, request, redirect
from PIL import Image
import numpy as np
import imagehash
from io import BytesIO
from pyngrok import ngrok
import os

app = Flask(__name__)

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/upload', methods=['POST'])
def upload():
    try:
        if 'file' not in request.files:
            return "No file uploaded", 400

        file = request.files['file']

        if file.filename == '':
            return "No file selected", 400

        # Read image file
        image = Image.open(file)
        # Preprocess image
        image = image.resize((128, 128))
        image = np.array(image)
        # Compute image hash
        hash_str = imagehash.average_hash(Image.fromarray(image))
        hash_hex = hash_str.hex()

        # Embed hash into the image
        embedded_image = Image.open(BytesIO(file.read()))
        embedded_image.save('static/embedded_image.jpeg')

        return render_template('result.html', hash_hex=hash_hex)

    except Exception as e:
        return f"An error occurred: {str(e)}", 500

# Install dependencies
!pip install flask pillow pyngrok

# Define HTML templates as strings
index_html = """
<!DOCTYPE html>
<html>
<head>
    <title>Upload Image</title>
</head>
<body>
    <h1>Upload Image</h1>
    <form action="/upload" method="post" enctype="multipart/form-data">
        <input type="file" name="file" accept="image/*">
        <input type="submit" value="Upload">
    </form>
</body>
</html>
"""

result_html = """
<!DOCTYPE html>
<html>
<head>
    <title>Result</title>
</head>
<body>
    <h1>Image Processing Result</h1>
    <p>Hash code: {{ hash_hex }}</p>
    <img src="/static/embedded_image.jpeg" alt="Embedded Image">
</body>
</html>
"""

# Create templates directory if it doesn't exist
if not os.path.exists('templates'):
    os.makedirs('templates')

# Create HTML template files
with open('templates/index.html', 'w') as file:
    file.write(index_html)

with open('templates/result.html', 'w') as file:
    file.write(result_html)

# Start ngrok tunnel
public_url = ngrok.connect(addr='5000', proto='http')
print('Public URL:', public_url)

# Run Flask app
app.run()


Public URL: NgrokTunnel: "https://3eb4-34-16-24-115.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [20/Apr/2024 09:06:53] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Apr/2024 09:06:53] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [20/Apr/2024 09:07:00] "POST /upload HTTP/1.1" 500 -
INFO:werkzeug:127.0.0.1 - - [20/Apr/2024 09:07:46] "POST /upload HTTP/1.1" 500 -
INFO:werkzeug:127.0.0.1 - - [20/Apr/2024 09:07:52] "POST /upload HTTP/1.1" 500 -


In [12]:
# Define Flask application
from flask import Flask, render_template, request, send_file
from PIL import Image
import numpy as np
import imagehash
from io import BytesIO
from pyngrok import ngrok
import os

app = Flask(__name__)

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/upload', methods=['POST'])
def upload():
    try:
        if 'file' not in request.files:
            return "No file uploaded", 400

        file = request.files['file']

        if file.filename == '':
            return "No file selected", 400

        # Read image file
        image = Image.open(file.stream)
        # Preprocess image
        image = image.resize((128, 128))
        # Compute image hash
        hash_str = imagehash.average_hash(image)
        hash_hex = str(hash_str)

        # Embed hash into the image filename
        filename = f'static/embedded_image_{hash_hex}.jpeg'
        # Save the original image with the embedded hash in the filename
        image.save(filename)

        return render_template('result.html', hash_hex=hash_hex, filename=filename)

    except Exception as e:
        return f"An error occurred: {str(e)}", 500


# Define HTML templates as strings
index_html = """
<!DOCTYPE html>
<html>
<head>
    <title>Upload Image</title>
</head>
<body>
    <h1>Upload Image</h1>
    <form action="/upload" method="post" enctype="multipart/form-data">
        <input type="file" name="file" accept="image/*">
        <input type="submit" value="Upload">
    </form>
</body>
</html>
"""

result_html = """
<!DOCTYPE html>
<html>
<head>
    <title>Result</title>
</head>
<body>
    <h1>Image Processing Result</h1>
    <p>Hash code: {{ hash_hex }}</p>
    <img src="{{ filename }}" alt="Embedded Image">
    <p><a href="{{ filename }}" download>Download Embedded Image</a></p>
</body>
</html>
"""

# Create templates directory if it doesn't exist
if not os.path.exists('templates'):
    os.makedirs('templates')

# Create HTML template files
with open('templates/index.html', 'w') as file:
    file.write(index_html)

with open('templates/result.html', 'w') as file:
    file.write(result_html)

# Start ngrok tunnel
public_url = ngrok.connect(addr='5000', proto='http')
print('Public URL:', public_url)

# Run Flask app
app.run()


Public URL: NgrokTunnel: "https://1804-34-16-24-115.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [20/Apr/2024 09:30:22] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Apr/2024 09:30:22] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [20/Apr/2024 09:30:29] "POST /upload HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Apr/2024 09:30:29] "GET /static/embedded_image_000000ffb7ffff4f.jpeg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Apr/2024 09:30:54] "GET /static/embedded_image_000000ffb7ffff4f.jpeg HTTP/1.1" 200 -
